In [1]:
import tensorflow as tf
tf.test.is_built_with_cuda()
tf.config.list_physical_devices('GPU') 

2021-12-11 17:11:23.158141: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

time: 1.16 s (started: 2021-12-11 17:11:22 +00:00)


2021-12-11 17:11:23.158615: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 17:11:23.162433: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 17:11:23.162867: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 17:11:23.163328: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 17:11:23.163751: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2021-12-11 17:11:23.201935: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-11 17:11:23.361371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 17:11:23.361831: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 17:11:23.362259: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

True

time: 695 ms (started: 2021-12-11 17:11:23 +00:00)


In [3]:
import tensorflow as tf
import numpy as np
import scipy.io 
from sklearn.preprocessing import StandardScaler
train = scipy.io.loadmat('train.mat') 
train_x = np.concatenate((train['x1'],train['x2']), axis=1)
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
train_y = train['y']
test = scipy.io.loadmat('test.mat') 
test_x = np.concatenate((test['x1'],test['x2']), axis=1)
test_x = scaler.transform(test_x)
test_y = test['y']

class DNN(tf.keras.Model):
  def __init__(self):
    super(DNN,self).__init__()
    self.dense1 = tf.keras.layers.Dense(10, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)
    self.dense3 = tf.keras.layers.Dense(10, activation=tf.nn.relu)
    self.output_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    x = self.dense3(x)
    return self.output_layer(x)


batch_size = 10
epochs = 1000
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model = DNN()
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=0)
model.summary()

score = model.evaluate(test_x, test_y, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

2021-12-11 17:11:24.050427: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 17:11:24.050887: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 17:11:24.051301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 17:11:24.051700: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-11 17:11:24.052103: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

Model: "dnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  30        
                                                                 
 dense_1 (Dense)             multiple                  110       
                                                                 
 dense_2 (Dense)             multiple                  110       
                                                                 
 dense_3 (Dense)             multiple                  11        
                                                                 
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________
Test loss: 0.1888955682516098
Test accuracy: 0.9666666388511658
time: 6.96 s (started: 2021-12-11 17:11:23 +00:00)


In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

train_data = pd.read_csv('train_DefenseSystem.csv', parse_dates=['event_time'], infer_datetime_format=True)
test_data = pd.read_csv('test_DefenseSystem.csv', parse_dates=['event_time'], infer_datetime_format=True) # test_data has no lable

lenc = LabelEncoder()
train_y = lenc.fit_transform(train_data['event_rule_category'])
train_y = train_y.reshape(-1,1)
pre_train_x = train_data.drop(['event_rule_category'], axis = 1)
pre_test_x = test_data
display(pre_train_x.shape, pre_test_x.shape)
display(pre_train_x.head())
display(pre_train_x.info())
display(pre_train_x.describe())
display(np.unique(train_y), lenc.classes_, train_y.shape)

(5000, 17)

(1020, 17)

,device_dev_name,device_family_name,device_hashed_mac,device_os_name,device_type_name,device_vendor_name,event_protocol_id,event_flow_outbound_or_inbound,event_role_device_or_router,event_role_server_or_client,event_rule_id,event_rule_name,event_rule_reference,event_rule_severity,event_self_ipv4,event_time,router_ip
0,Windows XP,Computer,a01f58b5c4fdb3122d6797ada0a3ee71fbe134dd,Windows XP,Desktop/Laptop,Microsoft Corp.,6,outbound,device,client,1050015,WEB Cross-site Scripting -34,CVE-2011-2133; CVE-2014-4116,4,192.168.1.243,2016-12-15 03:57:00,161.139.40.242
1,Windows,Computer,7c16de1f49ef32f5354a33ccd2ec5d0d5ef9e100,Windows,Desktop/Laptop,Microsoft Corp.,6,outbound,device,client,1130593,WEB Microsoft IIS HTTP.sys Remote Code Executi...,CVE-2015-1635; MS15-034,5,192.168.1.133,2016-12-11 06:54:00,199.221.3.43
2,Windows 8/10,Computer,6f8cdf9d553e2e1cba4c3b0ad226e6c7df119005,Windows 8/10,Desktop/Laptop,Microsoft Corp.,6,outbound,device,client,1056078,WEB Hashtable Collisions,CVE-2011-3414; CVE-2011-5034; CVE-2011-5035; C...,5,192.168.1.100,2016-12-24 01:20:00,157.2.60.102
3,Windows,Computer,4ac93ab7639e8800779afd887d6df7a93deb5c62,Windows,Desktop/Laptop,Microsoft Corp.,6,outbound,device,client,1050015,WEB Cross-site Scripting -34,CVE-2011-2133; CVE-2014-4116,4,192.168.1.6,2016-12-31 07:22:00,51.20.54.170
4,Windows,Computer,02de4773b0099f5b0a873516d76a888c3f4c6508,Windows,Desktop/Laptop,Microsoft Corp.,6,outbound,device,client,1050015,WEB Cross-site Scripting -34,CVE-2011-2133; CVE-2014-4116,4,192.168.1.44,2016-12-19 12:22:00,116.95.157.235


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   device_dev_name                 5000 non-null   object        
 1   device_family_name              5000 non-null   object        
 2   device_hashed_mac               5000 non-null   object        
 3   device_os_name                  5000 non-null   object        
 4   device_type_name                5000 non-null   object        
 5   device_vendor_name              5000 non-null   object        
 6   event_protocol_id               5000 non-null   int64         
 7   event_flow_outbound_or_inbound  5000 non-null   object        
 8   event_role_device_or_router     5000 non-null   object        
 9   event_role_server_or_client     5000 non-null   object        
 10  event_rule_id                   5000 non-null   int64         
 11  even

None

,event_protocol_id,event_rule_id,event_rule_severity
count,5000.000000,5.000000e+03,5000.000000
mean,6.410200,1.080985e+06,4.363600
std,2.089308,3.626082e+04,0.481083
min,0.000000,1.049802e+06,4.000000
25%,6.000000,1.050015e+06,4.000000
50%,6.000000,1.057007e+06,4.000000
75%,6.000000,1.130593e+06,5.000000
max,17.000000,1.132896e+06,5.000000


array([0, 1])

array(['Access Control', 'Web Attack'], dtype=object)

(5000, 1)

time: 49.7 ms (started: 2021-12-11 17:23:18 +00:00)


In [14]:
a_pre_data = pd.concat([pre_train_x,pre_test_x], axis=0)
a_pre_data = a_pre_data.drop(['device_hashed_mac'], axis=1)
for k in pre_train_x.keys().drop(['device_hashed_mac']):
    if k == 'event_self_ipv4': 
        tmp = a_pre_data['event_self_ipv4'].str.extract(r'(.*?)\.(.*?)\.(.*?)\.(.*)')
        a_pre_data['self_ip_class_A'] = (tmp[0]=='10').astype(int)
        a_pre_data['self_ip_class_B'] = (tmp[0]=='172').astype(int)
        a_pre_data['self_ip_class_B'] = (tmp[0]=='192').astype(int)
        a_pre_data['self_ip_others'] = ((tmp[0] != '192') * (tmp[0] != '172') * (tmp[0] != '10') == True).astype(int) # ip not start with 10, 172,192 => convert to 1
        a_pre_data = a_pre_data.drop(['event_self_ipv4'], axis=1)
    elif k == 'router_ip':
        tmp = a_pre_data['router_ip'].str.extract(r'(.*?)\.(.*?)\.(.*?)\.(.*)')
        tmp[tmp[0].isnull()] = -1
        a_pre_data['router_ip_class_A'] = (np.logical_and((0<=tmp[0].astype(int)), (tmp[0].astype(int)<128))).astype(int)
        a_pre_data['router_ip_class_B'] = (np.logical_and((128<=tmp[0].astype(int)), (tmp[0].astype(int)<192))).astype(int)
        a_pre_data['router_ip_class_C'] = (np.logical_and((192<=tmp[0].astype(int)), (tmp[0].astype(int)<224))).astype(int)
        a_pre_data['router_ip_class_D'] = (np.logical_and((224<=tmp[0].astype(int)), (tmp[0].astype(int)<240))).astype(int)
        a_pre_data['router_ip_class_E'] = (np.logical_and((240<=tmp[0].astype(int)), (tmp[0].astype(int)<255))).astype(int)
        a_pre_data['router_ip_others'] = (tmp[0].astype(int) == -1).astype(int)
        a_pre_data = a_pre_data.drop(['router_ip'], axis=1)
    elif k == 'event_time':
        tmp = a_pre_data['event_time'].apply(lambda x : x.timestamp())
        a_pre_data['event_time'] = tmp 
    else:
        a_pre_data_one_hot =  pd.get_dummies(a_pre_data[k])
        a_pre_data = pd.concat([a_pre_data, a_pre_data_one_hot],axis=1)
        a_pre_data = a_pre_data.drop([k], axis=1)

train_x = a_pre_data[:len(pre_train_x)]
test_x = a_pre_data[len(pre_train_x):]
display(train_x.head())
display(train_x.info())
display(train_x.describe())
print(train_x.shape, test_x.shape)

,event_time,ASUS RT Router,Amazon Kindle,Android 3.0-4.1,Android 4.2-5.1,Android 6.0,Apple Watch/iPad Pro,Axis Network Camera,Belkin F5D8235-4 v2,BlackBerry,...,5,self_ip_class_A,self_ip_class_B,self_ip_others,router_ip_class_A,router_ip_class_B,router_ip_class_C,router_ip_class_D,router_ip_class_E,router_ip_others
0,1.481774e+09,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,1.481439e+09,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
2,1.482542e+09,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
3,1.483169e+09,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
4,1.482150e+09,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Columns: 292 entries, event_time to router_ip_others
dtypes: float64(1), int64(9), uint8(282)
memory usage: 1.8 MB


None

,event_time,ASUS RT Router,Amazon Kindle,Android 3.0-4.1,Android 4.2-5.1,Android 6.0,Apple Watch/iPad Pro,Axis Network Camera,Belkin F5D8235-4 v2,BlackBerry,...,5,self_ip_class_A,self_ip_class_B,self_ip_others,router_ip_class_A,router_ip_class_B,router_ip_class_C,router_ip_class_D,router_ip_class_E,router_ip_others
count,5.000000e+03,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,1.481885e+09,0.000800,0.00060,0.000800,0.060600,0.010800,0.003400,0.000400,0.000800,0.002000,...,0.363600,0.038400,0.926600,0.025000,0.279000,0.495200,0.174600,0.012400,0.011000,0.027800
std,7.754209e+05,0.028276,0.02449,0.028276,0.238619,0.103371,0.058216,0.019998,0.028276,0.044681,...,0.481083,0.192179,0.260818,0.156141,0.448552,0.500027,0.379663,0.110674,0.104313,0.164416
min,1.480551e+09,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.481210e+09,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.481853e+09,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.482570e+09,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,1.483229e+09,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


(5000, 292) (1020, 292)
time: 407 ms (started: 2021-12-11 17:23:24 +00:00)


In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x.values)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y)
test_x = scaler.transform(test_x.values)

time: 14.6 ms (started: 2021-12-11 17:23:28 +00:00)


In [16]:
class DNN_base(tf.keras.Model):

  def __init__(self, nn_width):
    super().__init__()
    self.dense1 = tf.keras.layers.Dense(nn_width, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(nn_width, activation=tf.nn.relu)
    self.dense3 = tf.keras.layers.Dense(nn_width, activation=tf.nn.relu)
    self.dense4 = tf.keras.layers.Dense(nn_width, activation=tf.nn.relu)
    self.dense5 = tf.keras.layers.Dense(nn_width, activation=tf.nn.relu)
    self.output_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    x = self.dense5(x)
    return self.output_layer(x)


batch_size = 100
epochs = 30
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
for w in [2, 10, 20]:
    model = DNN_base(w)
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
    model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=0)
    model.summary()
    model.save(f'./dnn_width{w}', save_format='tf')
    score = model.evaluate(valid_x, valid_y, verbose=0)
    print(f"When DNN width = {w}, Validation loss:{score[0]}, Validation accuracy:{score[1]}\n\n")

Model: "dnn_base_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            multiple                  586       
                                                                 
 dense_23 (Dense)            multiple                  6         
                                                                 
 dense_24 (Dense)            multiple                  6         
                                                                 
 dense_25 (Dense)            multiple                  6         
                                                                 
 dense_26 (Dense)            multiple                  6         
                                                                 
 dense_27 (Dense)            multiple                  3         
                                                                 
Total params: 613
Trainable params: 613
Non-trainable pa

In [17]:
import pandas as pd

load_model = tf.keras.models.load_model('./dnn_width20')
pred = load_model.predict(test_x)
pred = np.where(pred<0.5, 0, 1)
pred_str = lenc.inverse_transform(pred)
print(f'predition of test data: {pred_str}')
pd.DataFrame(pred_str, columns=['prediction']).to_csv('q2.csv')

predition of test data: ['Access Control' 'Web Attack' 'Web Attack' ... 'Access Control'
 'Web Attack' 'Web Attack']
time: 235 ms (started: 2021-12-11 17:28:15 +00:00)


/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
from tensorflow.dat